In [1]:
#manipulação de dados
import pandas as pd
import numpy as np

#automação
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#web scraping
import requests
from bs4 import BeautifulSoup

#string e data
import unidecode
from datetime import datetime, time

#tempo
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)

In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Downloading: 19.2kB [00:00, 2.44MB/s]                                                                          
[WDM] - Downloading: 19.2kB [00:00, ?B/s]                                                                              
[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 1.59M/1.59M [00:00<00:00, 10.6MB/s]
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_4200\3429260821.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [4]:
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_UserName"]').send_keys('gabriel_s_anjos@yahoo.com') #e-mail
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_Password"]').send_keys('Gabriel2022!') #senha
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_btnLogin"]').click() #login

In [5]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [6]:
#click setor/profissional
browser.find_element("xpath", '//*[@id="s2id_autogen1"]/a/span[1]').click()
#click [profissional]
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()

# Capturando setores:

In [7]:
#click criar novo plantão
browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()

In [8]:
import time

In [9]:
inicio = time.time() #iniciando o cronometro

#Setor
browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()

setor_html = browser.find_element("xpath", '//*[@id="select2-drop"]/ul').get_attribute('outerHTML')
soup = BeautifulSoup(setor_html, 'html.parser')

fim = time.time() #encerrando cronometro
print('\nTempo que demorou o codigo:', fim - inicio)


Tempo que demorou o codigo: 0.8844208717346191


In [10]:
inicio = time.time() #iniciando o cronometro

lista_unidade = []
lista_unidade_count = []
lista_plt = []
lista_plt_count = []

count_unidade = 1

for i in soup.find_all(class_="select2-results-dept-0 select2-result select2-result-unselectable select2-result-with-children"):
    #print('\n', i.find(class_='select2-result-label').get_text(), '====== \n')
    count_plt = 1
    for t in i.find_all(class_='select2-results-dept-1 select2-result select2-result-selectable'): 
        #print(t.get_text())
        lista_plt.append(t.get_text())
        lista_plt_count.append(count_plt)
        count_plt += 1
            
        lista_unidade.append(i.find(class_='select2-result-label').get_text())
        lista_unidade_count.append(count_unidade)
    count_unidade += 1

scrap_unidades = pd.DataFrame({'unidade':lista_unidade, 'momento':lista_plt,
                               'unidade_count':lista_unidade_count, 'plt_count':lista_plt_count})

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: x.replace('  ',' '))

scrap_unidades['momento'] = scrap_unidades['momento'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
scrap_unidades['unidade'] = scrap_unidades['unidade'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))

fim = time.time() #encerrando cronometro
print('\nTempo que demorou o codigo:', fim - inicio)




Tempo que demorou o codigo: 0.04908561706542969


In [11]:
scrap_unidades.to_excel("setores.xlsx", index=False)
scrap_unidades.sample(20)

,unidade,momento,unidade_count,plt_count
89,JABAQUARA,SEMANA - NOTURNO - SAIDA,6,18
76,JABAQUARA,FDS - PLANTAO,6,5
57,ITAIM,FDS - D2,5,5
175,VILLA,SEMANA - NOTURNO - SAIDA,11,19
11,ADMINISTRACAO,PERIODO DE DESCANSO - 11,1,12
120,MORUMBI,SEMANA - TARDE - C.DIAGNOSTICO,8,18
166,VILLA,SEMANA - MANHA - AV.PRE ANESTESICA,11,10
65,ITAIM,SEMANA - MANHA - COBERTURA EXTRA,5,13
86,JABAQUARA,SEMANA - TARDE - COBERTURA EXTRA,6,15
132,OSASCO,SEMANA - 6HS,9,8


In [12]:
for i in scrap_unidades.columns:
    print(scrap_unidades[i].unique(),'\n')

['ADMINISTRACAO' 'ANALIA' 'AVICCENA' 'HOSPITAL GABRIEL DO ANJOS' 'ITAIM'
 'JABAQUARA' 'MATERNIDADE' 'MORUMBI' 'OSASCO' 'SANTA ISABEL' 'VILLA' 'VNS'] 

['MES ANTERIOR' 'PERIODO DE DESCANSO - 1' 'PERIODO DE DESCANSO - 2'
 'PERIODO DE DESCANSO - 3' 'PERIODO DE DESCANSO - 4'
 'PERIODO DE DESCANSO - 5' 'PERIODO DE DESCANSO - 6'
 'PERIODO DE DESCANSO - 7' 'PERIODO DE DESCANSO - 8'
 'PERIODO DE DESCANSO - 9' 'PERIODO DE DESCANSO - 10'
 'PERIODO DE DESCANSO - 11' 'PERIODO DE DESCANSO - 12'
 'PERIODO DE DESCANSO - 13' 'PERIODO DE DESCANSO - 14'
 'PERIODO DE DESCANSO - 15' 'PERIODO DE DESCANSO - 16' 'TESTE'
 'TREINAMENTO' 'FDS - 6HS' 'FDS - AVALIACAO PRE ROTINA'
 'FDS - C.CIRURGICO ROTINA' 'FDS - D1' 'FDS - D2'
 'FDS - D3 - PRIMEIRO (R)' 'FDS - D3 - SEGUNDO (B)' 'FDS - PLANTAO'
 'FDS - SAIDA' 'SEMANA - 6HS' 'SEMANA - MANHA - AV.PRE ANESTESICA'
 'SEMANA - MANHA - AMBULATORIO' 'SEMANA - MANHA - C.CIRURGICO'
 'SEMANA - MANHA - COBERTURA EXTRA' 'SEMANA - TARDE - AV.PRE ANESTESICA'
 'SEMANA - TARDE -

# TESTANDO SETORES

In [13]:
df = pd.read_excel('desloc_apagadook.xlsx')
df

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor
0,[29/06/2023 16:27:17] Marcelo Felipe de Souza ...,29/06/2023,MARCELO FELIPE DE SOUZA BUTO,29/06 OSASCO - BUTO SAIDA 16:20 ( DESLOCADO I...,CERTO,29/06/2023,OSASCO,2,DESLOCADO,NaN,SAIDA,16:30,semana,16:20:00,13:00,16:30,tarde,03:30,SEMANA - TARDE - C.CIRURGICO
1,[29/06/2023 17:16:59] Bianca Priscila Buniotti...,29/06/2023,BIANCA PRISCILA BUNIOTTI,29/06 ITAIM - BIANCA SAIDA 15:30 ( PÓS PLANTÃ...,CERTO,29/06/2023,ITAIM,2,DESLOCADO,NaN,SAIDA,15:30,semana,15:30:00,13:00,15:30,tarde,02:30,SEMANA - TARDE - C.CIRURGICO
2,[30/06/2023 05:17:59] Francisco Antonio Alves ...,30/06/2023,FRANCISCO ANTONIO ALVES BONATTO,30/06 ANÁLIA - FRANCISCO ENTRADA 05:10 ( DESL...,CERTO,30/06/2023,ANALIA,2,DESLOCADO,NaN,ENTRADA,05:00,semana,05:10:00,05:00,07:00,6h,02:00,SEMANA - 6HS
3,[30/06/2023 10:39:21] Gabriela Santos Menezes:...,30/06/2023,GABRIELA SANTOS MENEZES,30/06 OSASCO - G.MENEZES ENTRADA 06:20 ( DESL...,CERTO,30/06/2023,OSASCO,2,DESLOCADO,NaN,ENTRADA,06:00,semana,06:20:00,06:00,07:00,6h,01:00,SEMANA - 6HS


In [14]:
for X in range(0, len(df)):
    if len(scrap_unidades.loc[(scrap_unidades.unidade == df['unidade'][X]) & 
                                 (scrap_unidades.momento == df['setor'][X])]) < 1:
        print(X)
        print(df['unidade'][X], '-', df['setor'][X])
        print('index', ind)
        print(scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        print('codigos:', unidade, setorzin)
        
        break

print("="*20, "TUDO OK", "="*20)

==================== TUDO OK ====================


In [15]:
print(df['unidade'][X])
print(df['setor'][X])

OSASCO
SEMANA - 6HS


# RODANDO NO PEGA AS SAÍDAS

In [28]:
len(df)

4

In [29]:
H=0

In [31]:
for h in range(H, len(df)):
    print(h)
    #click criar novo plantão
    browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
    #esta disponivel o botão de setor
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
    #Setor====
    print('base:', df['unidade'][h], '-', df['setor'][h])
    browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
    #a
    
    #PROFISSIONAL
    #esta disponivel o botão de profissional
    WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))
    
    #listar todos
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()
    
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
    sleep(0.5)
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(df['nome'][h])
    #Dando enter
    browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
    #info
    print('base:', df['nome'][h])
    print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
    print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)
    
    sleep(2)
    #Data
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(df['data'][h])
    sleep(0.5)
    print(df['data'][h])
    
    #Inicio do plantão
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys(df['ENTRADA'][h])
    sleep(1)
    print('hora de entrada:', df['ENTRADA'][h], '\nhora de saida:', df['SAIDA'][h])
    #Duração do plantão
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
    browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys(df['DURAÇÃO'][h])
    print('duração do plantão', df['DURAÇÃO'][h])
        
    ##TIPO
    '''
    //*[@id="select2-drop"]/ul/li[3]/div
    1 - Normal
    2 - DESLOCAMENTO
    3 - VOLUNTÁRIO
    4 - COBERTURA
    '''
    if df['extra'][h] == 'extra':
        print('EXTRA')
        sleep(1)
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="s2id_add-service-service-types"]/a/span[1]').click()
        sleep(1)
        #selecionando o tipo voluntario:
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[3]/div').click()
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(200) #Pontos
            print('duração >5 -> 200 pontos')
        else:
            pass
    else:
        if int(df['DURAÇÃO'][h][:2]) >= 5 and df['sem_ou_fds'][h] == "semana":
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').clear()
            browser.find_element("xpath", '//*[@id="txtServiceValue"]').send_keys(100) #Pontos
            print('duração >5 -> 100 pontos')
        else:
            pass
        sleep(2)
        
    #salvar plantão
    sleep(1)
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
    sleep(4)

    #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
    if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
        #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
        sleep(1)
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        #
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
        df.loc[int(h), 'controle'] = "PROB"
        print("PROB")
    else:
        df.loc[int(h), 'controle'] = "NORM"
    
    #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

    #não esta disponivel o X
    
    WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
    H += 1
    print('='*40, '\n')

0
base: OSASCO - SEMANA - TARDE - C.CIRURGICO


ElementClickInterceptedException: Message: Element <input id="service-ckb-list-all-owner" class="pp-c-checkbox--input" type="checkbox"> is not clickable at point (779,553) because another element <div id="select2-drop-mask" class="select2-drop-mask"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:182:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:281:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:160:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:119:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:198:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:86:31


#CRIANDO PERÍODOS 7-13H PARA QUEM SAIU DEPOIS 13H

In [32]:
saida = df[df['tipo'] == 'SAIDA']
saida.reset_index(inplace=True, drop=True)

In [33]:
import datetime as dt  

In [34]:
saida['dt'] = pd.to_datetime(saida['SAIDA'], format='%H:%M').dt.time

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_4200\2243467139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  saida['dt'] = pd.to_datetime(saida['SAIDA'], format='%H:%M').dt.time


In [35]:
def entre(inicio, fim, checar): 
    if inicio < fim: 
        return checar >= inicio and checar <= fim
    else: 
        #Over midnight: 
        return checar >= inicio or checar <= fim

In [36]:
H = 0 

In [ ]:
for h in range(0,len(saida)):
    if entre(dt.time(13,1), dt.time(19,0), saida['dt'][h]) is True:
        browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
        #esta disponivel o botão de setor
        WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
        #Setor====
        print('base:', saida['unidade'][h], '-', saida['setor'][h])
        browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
        ind = scrap_unidades.loc[(scrap_unidades.unidade == saida['unidade'][h]) & 
                                 (scrap_unidades.momento == saida['setor'][h])].index.values[0]
        unidade = scrap_unidades['unidade_count'][ind]
        setorzin = scrap_unidades['plt_count'][ind]
        print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
        print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
              browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
        print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')

        #PROFISSIONAL
        #esta disponivel o botão de profissional
        WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))

        #listar todos
        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()

        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
        sleep(0.5)
        browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(saida['nome'][h])
        #Dando enter
        browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
        #info
        print('base:', saida['nome'][h])
        print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
        print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)

        sleep(2)
        #Data
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
        sleep(1)
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(saida['data'][h])
        sleep(0.5)
        print(saida['data'][h])

        #Inicio do plantão
        browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
        browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys('07:00')
        sleep(1)
        print('hora de entrada 13h e saída 19h')
        #Duração do plantão
        browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
        browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys('06:00')
        print('duração do plantão 6h')
        #Valor do plantão
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').click()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').click()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').clear()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').send_keys('1,00')
        print('valor do plantao 1,00')
        sleep(0.5)
        ##TIPO
        '''
        //*[@id="select2-drop"]/ul/li[3]/div
        1 - Normal
        2 - DESLOCAMENTO
        3 - VOLUNTÁRIO
        4 - COBERTURA
        '''

        #salvar plantão
        sleep(1)
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        sleep(4)

        #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
        if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
            #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
            sleep(1)
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            #
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
            saida.loc[int(h), 'controle'] = "PROB"
            print("PROB")
        else:
            saida.loc[int(h), 'controle'] = "NORM"

        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

        #não esta disponivel o X

        WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
        H += 1
        print('='*40, '\n')
        
    elif entre(dt.time(19,1), dt.time(3,0), saida['dt'][h]) is True:
        browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
        #esta disponivel o botão de setor
        WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
        #Setor====
        print('base:', saida['unidade'][h], '-', saida['setor'][h])
        browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
        ind = scrap_unidades.loc[(scrap_unidades.unidade == saida['unidade'][h]) & 
                                 (scrap_unidades.momento == saida['setor'][h])].index.values[0]
        unidade = scrap_unidades['unidade_count'][ind]
        setorzin = scrap_unidades['plt_count'][ind]
        print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
        print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
              browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
        print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')

        #PROFISSIONAL
        #esta disponivel o botão de profissional
        WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))

        #listar todos
        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()

        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
        sleep(0.5)
        browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(saida['nome'][h])
        #Dando enter
        browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
        #info
        print('base:', saida['nome'][h])
        print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
        print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)

        sleep(2)
        #Data
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
        sleep(1)
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(saida['data'][h])
        sleep(0.5)
        print(saida['data'][h])

        #Inicio do plantão
        browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
        browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys('07:00')
        sleep(1)
        print('hora de entrada 13h e saída 19h')
        #Duração do plantão
        browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
        browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys('06:00')
        print('duração do plantão 6h')
        #Valor do plantão
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').click()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').click()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').clear()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').send_keys('1,00')
        print('valor do plantao 1,00')
        sleep(0.5)
        ##TIPO
        '''
        //*[@id="select2-drop"]/ul/li[3]/div
        1 - Normal
        2 - DESLOCAMENTO
        3 - VOLUNTÁRIO
        4 - COBERTURA
        '''

        #salvar plantão
        sleep(1)
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        sleep(4)

        #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
        if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
            #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
            sleep(1)
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            #
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
            saida.loc[int(h), 'controle'] = "PROB"
            print("PROB")
        else:
            saida.loc[int(h), 'controle'] = "NORM"

        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

        #não esta disponivel o X

        WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
        H += 1
        print('='*40, '\n')

base: OSASCO - SEMANA - TARDE - C.CIRURGICO
scraping: OSASCO - SEMANA - TARDE - C.CIRURGICO
caixa: Osasco - Semana - Tarde - C.Cirúrgico
selecionado: Semana - Tarde - C.Cirúrgico 

base: MARCELO FELIPE DE SOUZA BUTO
selecionado: MARCELO FELIPE DE SOUZA BUTO



#CRIANDO PERÍODO 13-19H PARA QUEM SAIU DEPOIS DAS 19H

In [37]:
H = 0

In [38]:
for h in range(0,len(saida)):
    if entre(dt.time(19,1), dt.time(3,00), saida['dt'][h]) is True:
        browser.find_element("xpath", '//*[@id="shift-monthly"]/div[2]/table/tbody/tr[1]/td[4]/button').click()
    
        #esta disponivel o botão de setor
        WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]')))
        #Setor====
        print('base:', saida['unidade'][h], '-', saida['setor'][h])
        browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').click()
        ind = scrap_unidades.loc[(scrap_unidades.unidade == saida['unidade'][h]) & 
                                 (scrap_unidades.momento == saida['setor'][h])].index.values[0]
        unidade = scrap_unidades['unidade_count'][ind]
        setorzin = scrap_unidades['plt_count'][ind]
        print('scraping:', scrap_unidades['unidade'][ind], '-', scrap_unidades['momento'][ind])
        #SELECIONANDO Setor ##//*[@id="select2-drop"]/ul/li[unidade]/ul/li[momento]/div
        print('caixa:', browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/div').text, '-',
              browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').text)
        browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li['+str(unidade)+']/ul/li['+str(setorzin)+']/div').click()
        print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlServiceSector"]/a/span[1]').text, '\n')

        #PROFISSIONAL
        #esta disponivel o botão de profissional
        WebDriverWait(browser, 10).until(visibility_of(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]')))

        #listar todos
        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click()

        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
        browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').click() #########
        sleep(0.5)
        browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').click()
        browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(saida['nome'][h])
        #Dando enter
        browser.find_element("xpath", '//*[@id="select2-drop"]/div/input').send_keys(Keys.RETURN)
        #info
        print('base:', saida['nome'][h])
        print('selecionado:', browser.find_element("xpath", '//*[@id="s2id_ddlProfessionalA"]/a/span[1]').text)
        print(browser.find_element("xpath", '//*[@id="service-ckb-list-all-owner"]').text)

        sleep(2)
        #Data
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').clear()
        sleep(1)
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[2]/div[3]/div[5]/div[1]/div[2]/div/div[2]/div[1]/div[2]/input[1]').send_keys(saida['data'][h])
        sleep(0.5)
        print(saida['data'][h])

        #Inicio do plantão
        browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').clear()
        browser.find_element("xpath", '//*[@id="txtServiceStartTime"]').send_keys('13:00')
        sleep(1)
        print('hora de entrada 13h e saída 19h')
        #Duração do plantão
        browser.find_element("xpath", '//*[@id="txtServiceDuration"]').clear()
        browser.find_element("xpath", '//*[@id="txtServiceDuration"]').send_keys('06:00')
        print('duração do plantão 6h')
        #Valor do plantão
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').click()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').click()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').clear()
        browser.find_element(By.XPATH, '//*[@id="txtServiceValue"]').send_keys('1,00')
        print('valor do plantao 1,00')
        sleep(0.5)
        ##TIPO
        '''
        //*[@id="select2-drop"]/ul/li[3]/div
        1 - Normal
        2 - DESLOCAMENTO
        3 - VOLUNTÁRIO
        4 - COBERTURA
        '''

        #salvar plantão
        sleep(1)
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
        sleep(4)

        #VERIFICANDO SE HOUVE ERRO - HÁ PLANTÃO NO MESMO HORARIO
        if browser.find_element("xpath", '/html/body/div[5]/div/div[2]/div[2]/div/label').text != '':
            #browser.find_element("xpath", '//*[@id="ckb-add-service-ignore-conflict"]').click()#estou ciente
            sleep(1)
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click() #salvar plantão
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/div[2]/button[4]').click()
            #
            browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #FECHAR
            saida.loc[int(h), 'controle'] = "PROB"
            print("PROB")
        else:
            saida.loc[int(h), 'controle'] = "NORM"

        #browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button').click() #fechar {X}

        #não esta disponivel o X

        WebDriverWait(browser, 10).until_not(visibility_of(browser.find_element("xpath", '//*[@id="modal-add-service"]/div/div[1]/button')))
        H += 1
        print('='*40, '\n')

In [39]:
print(saida['controle'].value_counts())

NORM    14
PROB     1
Name: controle, dtype: int64


In [40]:
saida

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,DURAÇÃO,setor,dt
0,[26/05/2023 15:51:51] Denio Lopes de Almeida: ...,26/05/2023,DENIO LOPES DE ALMEIDA,26/05 OSASCO - DÊNIO SAIDA 15:50 ( PÓS PLANTÃ...,NORM,26/05/2023,OSASCO,2,DESLOCADO,NaN,SAIDA,16:00,semana,15:50:00,13:00,16:00,tarde,03:00,SEMANA - TARDE - C.CIRURGICO,16:00:00
1,[26/05/2023 16:15:36] Pedro Henrique Milhomens...,26/05/2023,PEDRO HENRIQUE MILHOMENS DA MOTA,26/05 MORUMBI MOTA SAIDA 16:00 ( DESLOCADO VN...,NORM,26/05/2023,MORUMBI,2,DESLOCADO,NaN,SAIDA,16:00,semana,16:00:00,13:00,16:00,tarde,03:00,SEMANA - TARDE - C.CIRURGICO,16:00:00
2,[27/05/2023 12:41:25] Arthur Rodrigues Torréli...,27/05/2023,ARTHUR RODRIGUES TORRELIO,27/05 ITAIM - TORRELIO SAIDA 12:35 ( PÓS ITA...,NORM,27/05/2023,ITAIM,3,DESLOCADO,NaN,SAIDA,13:00,fds,12:35:00,13:00,13:00,tarde,00:00,FDS - SAIDA,13:00:00
3,[27/05/2023 13:09:46] Sheila Itiroco Monte: 27...,27/05/2023,SHEILA ITIROCO MONTE,27/05 VNS SHEILA ( DESLOCADO ITAIM ) SAIDA 1...,NORM,27/05/2023,VNS,2,DESLOCADO,NaN,SAIDA,13:30,fds,13:09:00,13:00,13:30,tarde,00:30,FDS - SAIDA,13:30:00
4,[27/05/2023 21:30:49] Turibio Rodrigues Freire...,27/05/2023,TURIBIO RODRIGUES FREIRE NETO,27/05 OSASCO TURÍBIO ( DESLOCADO ITAIM ) SA...,NORM,27/05/2023,OSASCO,2,DESLOCADO,NaN,SAIDA,13:00,fds,12:50:00,13:00,13:00,tarde,00:00,FDS - SAIDA,13:00:00
5,[28/05/2023 17:13:22] Felipe de Oliveira Nunes...,28/05/2023,FELIPE DE OLIVEIRA NUNES,28/05 MORUMBI - NUNES SAIDA 11:15 ( D2 JABAQU...,PROB,28/05/2023,MORUMBI,2,DESLOCADO,NaN,SAIDA,11:30,fds,11:15:00,07:00,11:30,manha,04:30,FDS - C.CIRURGICO ROTINA,11:30:00
6,[29/05/2023 10:41:51] Leticia Fernandes Borges...,29/05/2023,LETICIA FERNANDES BORGES,29/05- MORUMBI- LETÍCIA SAIDA 10:40 ( DESLOC...,NORM,29/05/2023,MORUMBI,2,DESLOCADO,NaN,SAIDA,11:00,semana,10:40:00,07:00,11:00,manha,04:00,SEMANA - MANHA - C.CIRURGICO,11:00:00
7,[29/05/2023 12:30:58] Karen Yumi Miyashiro: 29...,29/05/2023,KAREN YUMI MIYASHIRO,29/05 ANÁLIA - YUMI SAIDA 12:25 ( PÓS PLANTÃO...,NORM,29/05/2023,ANALIA,2,DESLOCADO,NaN,SAIDA,12:30,semana,12:25:00,07:00,12:30,manha,05:30,SEMANA - MANHA - C.CIRURGICO,12:30:00
8,[29/05/2023 14:58:53] Beatriz Antipou dos Sant...,29/05/2023,BEATRIZ ANTIPOU DOS SANTOS,29/05 ITAIM- ANTIPOU- SAIDA 14:55 ( DESLOCADO...,NORM,29/05/2023,ITAIM,2,DESLOCADO,NaN,SAIDA,15:00,semana,14:55:00,13:00,15:00,tarde,02:00,SEMANA - TARDE - C.CIRURGICO,15:00:00
9,[29/05/2023 17:15:54] Marcela Aparecida Correa...,29/05/2023,MARCELA APARECIDA CORREA MARTINS,29/05 JABAQUARA - MARCELA MARTINS SAIDA 16:55 ...,NORM,29/05/2023,JABAQUARA,2,DESLOCADO,NaN,SAIDA,17:00,semana,16:55:00,13:00,17:00,tarde,04:00,SEMANA - TARDE - C.CIRURGICO,17:00:00


# Salvando restante:

In [41]:
df['controle'].value_counts()

NORM    23
PROB     2
Name: controle, dtype: int64

In [42]:
# Registros com Problemas Corrigiveis ou Não
df_corrigir4 = df[df['controle'] == 'PROB']
df_corrigir4.reset_index(drop=True, inplace=True)
print("probs:", len(df_corrigir4))

# Registros Perfeitos ja feitos
df_norm = df[df['controle'] == 'NORM']
df_norm.reset_index(drop=True, inplace=True)
print("norm:", len(df_norm))

#salvando
df_norm.to_excel("desloc_jainserido.xlsx", index=False) 
df_corrigir4.to_excel("desloc_acorrigir4.xlsx", index=False)

probs: 2
norm: 23
